## Observations and Insights 

In [1]:
%matplotlib notebook
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np

# Study data files
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

In [2]:
# Read the mouse data and the study results. Load in csv file.
mouse_metadata_df= pd.read_csv(mouse_metadata_path)
mouse_metadata_df

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g)
0,k403,Ramicane,Male,21,16
1,s185,Capomulin,Female,3,17
2,x401,Capomulin,Female,16,15
3,m601,Capomulin,Male,22,17
4,g791,Ramicane,Male,11,16
...,...,...,...,...,...
244,z314,Stelasyn,Female,21,28
245,z435,Propriva,Female,12,26
246,z581,Infubinol,Female,24,25
247,z795,Naftisol,Female,13,29


In [3]:
# Read the mouse data and the study results. Load in csv file.
study_results_df = pd.read_csv(study_results_path)
study_results_df

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites
0,b128,0,45.000000,0
1,f932,0,45.000000,0
2,g107,0,45.000000,0
3,a457,0,45.000000,0
4,c819,0,45.000000,0
...,...,...,...,...
1888,r944,45,41.581521,2
1889,u364,45,31.023923,3
1890,p438,45,61.433892,1
1891,x773,45,58.634971,4


In [4]:
# Combine the data into a single dataset
combined_results_df=pd.merge(mouse_metadata_df,study_results_df,how="outer",on="Mouse ID")
combined_results_df

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,k403,Ramicane,Male,21,16,0,45.000000,0
1,k403,Ramicane,Male,21,16,5,38.825898,0
2,k403,Ramicane,Male,21,16,10,35.014271,1
3,k403,Ramicane,Male,21,16,15,34.223992,1
4,k403,Ramicane,Male,21,16,20,32.997729,1
...,...,...,...,...,...,...,...,...
1888,z969,Naftisol,Male,9,30,25,63.145652,2
1889,z969,Naftisol,Male,9,30,30,65.841013,3
1890,z969,Naftisol,Male,9,30,35,69.176246,4
1891,z969,Naftisol,Male,9,30,40,70.314904,4


In [5]:
# Checking the number of mice in the DataFrame.
        # mice_instances_combined=combined_results_df["Mouse ID"].count()
        # mice_instances_combined
mouse_metadata_df.count()

Mouse ID        249
Drug Regimen    249
Sex             249
Age_months      249
Weight (g)      249
dtype: int64

In [6]:
## DUPLICATE MOUSE IDENTIFIED ##

# Getting the duplicate mice by ID number that shows up for Mouse ID and Timepoint. 
duplicate_rows=combined_results_df[combined_results_df.duplicated()]
duplicate_rows

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
909,g989,Propriva,Female,21,26,0,45.0,0


In [7]:
## Optional: Get all the data for the duplicate mouse ID. ##

duplicate_rows=combined_results_df[combined_results_df.duplicated(keep=False)]
print("All Duplicate Rows based on all data columns is :")
print(duplicate_rows)

All Duplicate Rows based on all data columns is :
    Mouse ID Drug Regimen     Sex  Age_months  Weight (g)  Timepoint  \
908     g989     Propriva  Female          21          26          0   
909     g989     Propriva  Female          21          26          0   

     Tumor Volume (mm3)  Metastatic Sites  
908                45.0                 0  
909                45.0                 0  


In [8]:
# Checking the number of mice in the clean DataFrame before dropping duplicate records.
combined_results_df.count()

Mouse ID              1893
Drug Regimen          1893
Sex                   1893
Age_months            1893
Weight (g)            1893
Timepoint             1893
Tumor Volume (mm3)    1893
Metastatic Sites      1893
dtype: int64

In [9]:
## REMOVE THE DUPLICATE MOUSE/MICE ##

# Create a clean DataFrame by dropping the duplicate mouse by its ID. 

#### LESSON PANDAS DAY 2 -01 ####
        #clean_combined_results_df=combined_results_df.drop_duplicates(keep='first')
        #print('Duplicate records dropped :\n', clean_combined_results_df)

clean_combined_results_df=combined_results_df.drop_duplicates(inplace=True)
#print(clean_combined_results_df)

In [10]:
# Test to validate that the duplicate record is dropped from the dataset.
duplicate_rows=combined_results_df[combined_results_df.duplicated(keep=False)]
print("All Duplicate Rows based on all data columns is :")
print(duplicate_rows)

All Duplicate Rows based on all data columns is :
Empty DataFrame
Columns: [Mouse ID, Drug Regimen, Sex, Age_months, Weight (g), Timepoint, Tumor Volume (mm3), Metastatic Sites]
Index: []


In [11]:
# Checking the number of mice in the clean DataFrame.
combined_results_df.count()

Mouse ID              1892
Drug Regimen          1892
Sex                   1892
Age_months            1892
Weight (g)            1892
Timepoint             1892
Tumor Volume (mm3)    1892
Metastatic Sites      1892
dtype: int64

## Summary Statistics

In [12]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen
# This method is the most straighforward, creating multiple series and putting them all together at the end.

# For Tumor Volume only use necessary columns
tumor_volume_df=combined_results_df.loc[:,["Drug Regimen","Timepoint","Tumor Volume (mm3)"]]
tumor_volume_df

,Drug Regimen,Timepoint,Tumor Volume (mm3)
0,Ramicane,0,45.000000
1,Ramicane,5,38.825898
2,Ramicane,10,35.014271
3,Ramicane,15,34.223992
4,Ramicane,20,32.997729
...,...,...,...
1888,Naftisol,25,63.145652
1889,Naftisol,30,65.841013
1890,Naftisol,35,69.176246
1891,Naftisol,40,70.314904


In [13]:
# Generate a summary statistics table
drug_regimen_df=tumor_volume_df.groupby(["Drug Regimen"])
drug_regimen_df.describe()

Timepoint                                                     \
                 count       mean        std  min   25%   50%   75%   max   
Drug Regimen                                                                
Capomulin        230.0  21.565217  14.384994  0.0  10.0  20.0  35.0  45.0   
Ceftamin         178.0  19.747191  14.283969  0.0   5.0  20.0  30.0  45.0   
Infubinol        178.0  18.174157  13.473473  0.0   5.0  15.0  30.0  45.0   
Ketapril         188.0  19.707447  14.029935  0.0   5.0  20.0  30.0  45.0   
Naftisol         186.0  19.623656  14.184814  0.0   5.0  20.0  30.0  45.0   
Placebo          181.0  18.674033  13.890798  0.0   5.0  15.0  30.0  45.0   
Propriva         160.0  16.968750  13.447977  0.0   5.0  15.0  25.0  45.0   
Ramicane         228.0  21.425439  14.275720  0.0  10.0  20.0  35.0  45.0   
Stelasyn         181.0  19.226519  13.842710  0.0   5.0  20.0  30.0  45.0   
Zoniferol        182.0  19.368132  14.384679  0.0   5.0  15.0  30.0  45.0   

             Tumor Volume (mm3)                                             \
                          count       mean       std        min        25%   
Drug Regimen                                                                 
Capomulin                 230.0  40.675741  4.994774  23.343598  37.685933   
Ceftamin                  178.0  52.591172  6.268188  45.000000  47.208427   
Infubinol                 178.0  52.884795  6.567243  36.321346  47.312353   
Ketapril                  188.0  55.235638  8.279709  45.000000  48.232987   
Naftisol                  186.0  54.331565  8.134708  45.000000  47.285874   
Placebo                   181.0  54.033581  7.821003  45.000000  47.459053   
Propriva                  160.0  52.368318  6.502160  45.000000  47.107256   
Ramicane                  228.0  40.216745  4.846308  22.050126  36.674635   
Stelasyn                  181.0  54.233149  7.710419  45.000000  48.047139   
Zoniferol                 182.0  53.236507  6.966589  45.000000  47.337876   

                                               
                    50%        75%        max  
Drug Regimen                                   
Capomulin     41.557809  45.000000  48.158209  
Ceftamin      51.776157  56.801438  68.923185  
Infubinol     51.820584  57.314444  72.226731  
Ketapril      53.698743  60.870951  78.567014  
Naftisol      52.509285  59.963034  76.668817  
Placebo       52.288934  59.916934  73.212939  
Propriva      50.909965  56.259803  72.455421  
Ramicane      40.673236  45.000000  47.622816  
Stelasyn      52.431737  58.719297  75.123690  
Zoniferol     51.818479  57.954259  73.324432

In [14]:
## DRUG REGIMEN VS. TUMOR VOLUME & TIMEPOINT SUMMARY STATISTICS TABLE ##

# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen
tumor_volume_statistics=tumor_volume_df.groupby(["Drug Regimen","Timepoint"]).agg({"Tumor Volume (mm3)":["mean","median","var","std","sem"]})
tumor_volume_statistics

Tumor Volume (mm3)                                  \
                                     mean     median        var       std   
Drug Regimen Timepoint                                                      
Capomulin    0                  45.000000  45.000000   0.000000  0.000000   
             5                  44.266086  45.597064   5.030889  2.242964   
             10                 43.084291  43.421014  12.344133  3.513422   
             15                 42.064317  42.798160  16.878693  4.108369   
             20                 40.716325  40.716428  19.035028  4.362915   
...                                   ...        ...        ...       ...   
Zoniferol    25                 55.432935  55.676604   5.808348  2.410052   
             30                 57.713531  57.419744   9.601024  3.098552   
             35                 60.089372  60.365832  10.876760  3.297993   
             40                 62.916692  62.274543  13.958456  3.736102   
             45                 65.960888  65.684689  14.100301  3.755037   

                                  
                             sem  
Drug Regimen Timepoint            
Capomulin    0          0.000000  
             5          0.448593  
             10         0.702684  
             15         0.838617  
             20         0.909731  
...                          ...  
Zoniferol    25         0.602513  
             30         0.800043  
             35         0.881426  
             40         0.998515  
             45         1.003576  

[100 rows x 5 columns]

In [15]:
## DRUG REGIMEN VS. TUMOR VOLUME SUMMARY STATISTICS TABLE ##

# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen
tumor_volume_stats=tumor_volume_df.groupby(["Drug Regimen"]).agg({"Tumor Volume (mm3)":["mean","median","var","std","sem"]})
tumor_volume_stats

Tumor Volume (mm3)                                          
                           mean     median        var       std       sem
Drug Regimen                                                             
Capomulin             40.675741  41.557809  24.947764  4.994774  0.329346
Ceftamin              52.591172  51.776157  39.290177  6.268188  0.469821
Infubinol             52.884795  51.820584  43.128684  6.567243  0.492236
Ketapril              55.235638  53.698743  68.553577  8.279709  0.603860
Naftisol              54.331565  52.509285  66.173479  8.134708  0.596466
Placebo               54.033581  52.288934  61.168083  7.821003  0.581331
Propriva              52.368318  50.909965  42.278090  6.502160  0.514041
Ramicane              40.216745  40.673236  23.486704  4.846308  0.320955
Stelasyn              54.233149  52.431737  59.450562  7.710419  0.573111
Zoniferol             53.236507  51.818479  48.533355  6.966589  0.516398

## Bar Plots

In [16]:
# Generate a bar plot showing the number of mice per time point for each treatment throughout the course of the study using pandas. 


In [17]:
# Generate a bar plot showing the number of mice per time point for each treatment throughout the course of the study using pyplot.


## Pie Plots

In [18]:
# Generate a pie plot showing the distribution of female versus male mice using pandas
# LESSON 2 - 05 BikeTrippin
# Split up our data into groups based upon 'gender'
#gender_groups = bike_trips_df.groupby('gender')
# Find out how many of each gender took bike trips
#gender_trips = gender_groups['tripduration'].count()

In [19]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot

## Quartiles, Outliers and Boxplots

In [20]:
# Calculate the final tumor volume of each mouse across four of the most promising treatment regimens. Calculate the IQR and quantitatively determine if there are any potential outliers. 


In [21]:
# Generate a box plot of the final tumor volume of each mouse across four regimens of interest

## Line and Scatter Plots

In [22]:
# Generate a line plot of time point versus tumor volume for a mouse treated with Capomulin


In [23]:
# Generate a scatter plot of mouse weight versus average tumor volume for the Capomulin regimen



## Correlation and Regression

In [24]:
# Calculate the correlation coefficient and linear regression model 
# for mouse weight and average tumor volume for the Capomulin regimen
